In [54]:
import polars as pl
import pathlib
import os
import streamlit as st



data = (
    pl.read_csv("data/DnB/report (2).csv")
    .with_columns(
        pl.col("date").cast(pl.Date),
    )
)

# contain the supplier and customer name mapping
supplier_abb = pl.read_csv("data/Supplier Name Map.csv", separator='\t').rename({'abb': 'supplier_abb'})
customer_abb = pl.read_csv("data/Customer Name Map.csv", separator=',').rename({'abb': 'customer_abb'})

data = (
    data
    .join(supplier_abb, on="supplier", how="left")
    .join(customer_abb, on="customer", how="left")
).rename(
    {
        'supplier': 'supplier_name',
        'customer': 'customer_name',
        'supplier_abb': 'supplier',
        'customer_abb': 'customer',
    }
)

agg_data = (
    data
    .group_by("supplier", "customer", "product", maintain_order=True)
    .agg(
        pl.col("factory_nett").sum().alias("factory_nett"),
        pl.col("deduction").sum().alias("deduction"),
    )
    .filter(pl.col("product") == "001 - FFB")
)

pivot_data = agg_data.pivot(
    "customer",
    index="supplier",
    values="factory_nett",
    aggregate_function="first",
    maintain_order=True,
).fill_null(0).fill_null(0)

pivot_data = pivot_data.with_columns(
    TOTAL = pl.sum_horizontal(pivot_data.select(pl.exclude("supplier")))
)

deduction_totals = (
    agg_data
    .group_by("supplier", maintain_order=True)
    .agg(pl.col("deduction").sum().alias("DEDUCTION_TOTAL"))
)

pivot_data = (
    pivot_data
    .join(deduction_totals, on="supplier", how="left")
    .with_columns(
        NETT_TOTAL = pl.col("TOTAL") - pl.col("DEDUCTION_TOTAL")
        
    )
)

dirpath = pathlib.Path("data/export")
pivot_data.write_csv(dirpath / "jan2025_abb.csv")
    
